In [1]:
import re 
import requests
headers_Get = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
}

In [2]:
from IPython.display import HTML 
import Levenshtein as lv
import unidecode
from bs4 import BeautifulSoup

In [86]:
#Calidad
def quality (gsr, title, author, journal, publisher, rq=2):
    '''
    Añade parametros de calidad al diccionario gsr
    
    rq=N: redondea a el número de cifras N
    '''
    
    #title quality
    if title:
        qt=lv.ratio(unidecode.unidecode(title).lower(),
                    unidecode.unidecode(gsr['title']).lower())
        gsr['title_quality']=round(qt,rq)
    else:
        gsr['title_quality']=-1#None
        
    # authors quality 
    authors=author+', '+', '.join(coauthors)
    authors
    if author:
        qa=lv.ratio(unidecode.unidecode(author).lower(),
                    unidecode.unidecode(gsr['authors']).lower())
        gsr['authors_quality']=round(qa,rq)
    else:
        gsr['authors_quality']=-1 
    # journal quality 
    if journal:
        qj=lv.ratio(unidecode.unidecode(journal).lower(),
                    unidecode.unidecode(gsr['journal']).lower())
        gsr['journal_quality']=round(qj,rq)
    else:
        gsr['journal_quality']=-1
    #publisher quality
    if publisher:
        qp=lv.ratio(unidecode.unidecode(publisher).lower(),
                    unidecode.unidecode(gsr['publisher']).lower())
        gsr['publisher_quality']=round(qp,rq)
    else:
        gsr['publisher_quality']=-1
        
    return gsr

# Implementar aquí
def getmetadatos(r):
    from bs4 import BeautifulSoup
    import sys
    res=BeautifulSoup(r.text,"html.parser")
    
    # Check robots detection
    if res.find('div',{'class':"g-recaptcha"}):
        sys.exit('ERROR: reCAPTCHA activied')

    gsr={}
    try:
        fr=res.find_all('div',{'class':'gs_r gs_or gs_scl'})[0]
    except IndexError:
        fr=BeautifulSoup('',"html.parser")
        
    #para autor  
    gsa=fr.find('div', {'class':'gs_a'}) 
    if gsa:
        gsr['authors']=gsa.text.split('\xa0-')[0]
    else:
        gsr['authors']=''    
            
            
    #para título    
    gst=fr.find('h3',{'class':'gs_rt'})
    if gst:
        gsr['title']=gst.find_all('a')[0].text
    else:
        gsr['title']=''

        
    #para link de citas 
    try:
        gsc=fr.find_all('div',{'class':'gs_fl'})[1]
    except IndexError:
        gsc=''

    if gsc:
        try:
            gscc=gsc.find_all('a', {'href':re.compile('/scholar\?cites=')})[0]
        except IndexError:
            gscc=''
        try:
            gsr['cites']=int( gscc.text.split()[-1] )
        except IndexError:
            gsr['cites']=0
        gsr['cites_link']=gscc.attrs.get('href')
    else:
        gsr['cites']=''
    
    #para link pdf
    try:
        gsp=fr.find_all('div',{'class':'gs_or_ggsm'})[0]
    except IndexError:
        gsp=''
    if gsp:
        gspd=gsp.find('a').get('href') # ['href'] pude generar error
        gsr['pdf']=gspd
    else:
        gsr['pdf']=''
        
    #Year
    gsy=fr.find('div', {'class':'gs_a'})
    if gsy:
        gsr['year']=gsy.text.split('-')[-2].strip().split(',')[-1]
        gsr['year']=eval('year')
    else:
        gsr['year']=''
    
    #Journal 
    gsj=fr.find('div', {'class':'gs_a'})
    if gsj:
        gsr['journal']=gsj.text.split('-')[-2].strip().split(',')[-2].replace('arXiv\xa0…','').strip()
    else:
        gsr['journal']=''


    #publisher
    gsj=fr.find('div', {'class':'gs_a'})
    if gsj:
        gsr['publisher']=gsj.text.split('-')[-1].strip()
    else:
        gsr['publisher']=''


    #google scholar profile en: https://scholar.google.com/citations?hl=en&user=HERE
    try:
        gpa=fr.find_all('div',{'class':'gs_a'})[0]
    except IndexError:
        gpa=''
    if gpa:
        prf={}
        for pr in gpa.find_all('a'):
            key=pr.text
            value=pr.get('href').split('=')[1].split('&')[0]
            prf[key]=value
        gsr['profiles']=prf 
        
    #abstract
    try:
        gsab=fr.find_all('div',{'class':'gs_rs'})[0]
    except IndexError:
        gsab=''
    if gsab:
        gsr['abstract']=gsab.text.replace('\xa0…','')

    return gsr
             
       
def googlescholar(title='', author='', coauthors=[], doi='', year=0, publisher='',
                  journal='', volume='', issue='', pages=0, DEBUG=False):
    '''
    DEBUG: return full web page
    '''
    import sys
    gsr={}
    
    s = requests.Session()
    url='https://scholar.google.co.in/scholar_lookup?'
    
    nl=0
    if title:
        nl=nl+1
        url= url+'title={}'.format(title.replace(' ','+'))     
    if author:
        if nl: url= url+'&'
        nl=nl+1
        url= url+'author={}'.format(author.replace(' ','+'))
    if doi:
        if nl: url= url+'&'
        nl=nl+1    
        url= url+'doi={}'.format(doi)
    if year:
        if nl: url= url+'&'
        nl=nl+1
        url= url+'year={}'.format(year)
    if publisher:
        if nl: url= url+'&'
        nl=nl+1
        url= url+'publisher={}'.format(publisher.replace(' ','+'))
    if journal:
        if nl: url= url+'&'
        nl=nl+1     
        url= url+'journal={}'.format(journal.replace(' ','+'))
    if volume:
        if nl: url= url+'&'
        nl=nl+1
        url= url+'volume={}'.format(volume)
    if issue:
        if nl: url= url+'&'
        nl=nl+1
        url= url+'issue={}'.format(issue)   
    if pages:
        if nl: url= url+'&'
        nl=nl+1
        url= url+'pages={}'.format(pages)
    if coauthors:
        for i in coauthors :
            if nl: url= url+'&'
            nl=nl+1
            url= url+'author={}'.format(i.replace(' ','+'))        
            
    r=s.get(url,headers=headers_Get)
    #print(url)
          
    gsr=getmetadatos(r)
    
    #análizar calidad
    gsr=quality (gsr, title, author, journal, publisher, rq=2)
    
    return gsr,url


In [87]:
gsr,url=googlescholar(doi='10.4067/S0034-98872007000700016',
                      title='Analysis of electronic references in a volume of Revista Médica de Chile', 
                      author='',
                      coauthors=[],
                      year=0,
                      publisher='',
                      journal='',
                      volume='',
                      issue='',
                      pages='' 
                     )

In [88]:
url

'https://scholar.google.co.in/scholar_lookup?title=Analysis+of+electronic+references+in+a+volume+of+Revista+Médica+de+Chile&doi=10.4067/S0034-98872007000700016'

In [89]:
gsr

{'authors': '',
 'authors_quality': -1,
 'cites': '',
 'journal': '',
 'journal_quality': -1,
 'pdf': '',
 'publisher': '',
 'publisher_quality': -1,
 'title': '',
 'title_quality': 0.0,
 'year': ''}

In [82]:
if gscc.text: print(1)

# Generar un dataframe de resultados
A partir de una lista con DOIs y títulos (una cada 2 segundos)

In [40]:
import time 
import pandas as pd

In [41]:
lista={'DOI': ['10.4185/RLCS-66-2011-926-101-129', 
               '10.4067/S0718-09342008000200012', 
               '10.4067/S0718-40262013000500003', 
               '10.4067/S0034-98872007000700016', 
               '10.1371/journal.pntd.0004045'], 
        'Título': ['La investigación sobre Comunicación en España (1998-2007). Análisis de los artículos publicados en revistas científicas', 
                   'Interpretaciones de la atenuación en artículos de revisión de Odontología',
                   'Productividad y calidad metodológica de artículos clínicos en cirugía oral y maxilofacial en Chile: Período 2001-2012',
                   'Analysis of electronic references in a volume of Revista Médica de Chile',
                   'Human Genetic Ancestral Composition Correlates with the Origin of Mycobacterium leprae Strains in a Leprosy Endemic Population.']}
d=pd.DataFrame(lista)

In [42]:
d

,DOI,Título
0,10.4185/RLCS-66-2011-926-101-129,La investigación sobre Comunicación en España ...
1,10.4067/S0718-09342008000200012,Interpretaciones de la atenuación en artículos...
2,10.4067/S0718-40262013000500003,Productividad y calidad metodológica de artícu...
3,10.4067/S0034-98872007000700016,Analysis of electronic references in a volume ...
4,10.1371/journal.pntd.0004045,Human Genetic Ancestral Composition Correlates...


In [48]:
d.index[2:4]

RangeIndex(start=2, stop=4, step=1)

In [90]:
df=pd.DataFrame()
tiempo=20
ini=2
for i in d.index[ini:]:
    doi=d.loc[i,'DOI']
    title=d.loc[i,'Título']  
    
    gsr,url=googlescholar(doi=doi,title=title,author=author,coauthors=coauthors,year=year,publisher=publisher,journal=journal)    
    
    print (i)
    #TU codigo
    #gsr=....
    #Añanadir a gsr los datos de DOI, Título (opcional) a gsr 
    #Actualizar dataframe
    #salvar el datframe
    time.sleep(tiempo)

2
3
4


In [52]:
i=3
d.loc[i,'Título']  ,d.loc[i,'DOI']

('Analysis of electronic references in a volume of Revista Médica de Chile',
 '10.4067/S0034-98872007000700016')

In [ ]:
df=pd.DataFrame()

for i in d.index:
    doi=d.loc[i,'DOI']
    title=d.loc[i,'Título']  
    
    gsr,url=googlescholar(doi=doi,title=title,author=author,coauthors=coauthors,year=year,publisher=publisher,journal=journal)    
    
    print (gsr)
    #TU codigo
    #gsr=....
    #Añanadir a gsr los datos de DOI, Título (opcional) a gsr 
    #Actualizar dataframe
    #salvar el datframe
    time.sleep(2)

In [22]:
lista={'DOI': ['10.4185/RLCS-66-2011-926-101-129', 
               '10.4067/S0718-09342008000200012'], 
        'Título': ['La investigación sobre Comunicación en España (1998-2007). Análisis de los artículos publicados en revistas científicas', 
                   'Interpretaciones de la atenuación en artículos de revisión de Odontología']}
d=pd.DataFrame(lista)

In [164]:
d

,DOI,Título
0,10.4185/RLCS-66-2011-926-101-129,La investigación sobre Comunicación en España ...
1,10.4067/S0718-09342008000200012,Interpretaciones de la atenuación en artículos...


In [165]:
df=pd.DataFrame()

for i in d.index:
    doi=d.loc[i,'DOI']
    title=d.loc[i,'Título']  
    
    gsr,url=googlescholar(doi=doi,title=title,author=author,coauthors=coauthors,year=year,publisher=publisher,journal=journal)    
    
    print (gsr)
    #TU codigo
    #gsr=....
    #Añanadir a gsr los datos de DOI, Título (opcional) a gsr 
    #Actualizar dataframe
    #salvar el datframe
    time.sleep(2)

IndexError: list index out of range